In [ ]:
library(phastCons100way.UCSC.hg38)
phast <- phastCons100way.UCSC.hg38

In [ ]:
wd = '/data/work/Brain/project/ATAC/humanPFC/00.peak_file/'

In [ ]:
cre_detail <- readRDS("human_peakSet.rds")

In [ ]:
# bgdpeak used in this part are from https://screen.encodeproject.org/
bdgpeak <- read.table("bgdPeaks.bed",sep='\t')
bdgpeak$V4="bgd"
bdgpeak=bdgpeak[which(bdgpeak$V1 %in% c('chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10','chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chr21','chr22','chrX','chrY')),]
bgd_GR=data.frame(seqnames=rep(bdgpeak$V1), start=bdgpeak$V2,end=bdgpeak$V3, strand=rep(strand("*")))
bgd_GR <- makeGRangesFromDataFrame(bgd_GR, keep.extra.columns=TRUE,na.rm=TRUE)
mcols(bgd_GR)$state <- 'bgdPeaks'
bgd_GR$peakName <- (bgd_GR %>% {paste0(seqnames(.), "_", start(.), "_", end(.))})

allPeaksGR=c(cre_detail,bgd_GR)
full_cre <- cre_detail
full_cre$state="all_cCREs"
allPeaksGR=c(allPeaksGR,full_cre)
filt_full_p2gGR=c(cre_detail,full_cre)

In [ ]:
allPeaksCons <- gscores(phast, allPeaksGR, summaryFun=mean) # Mean is the default summaryFun

# Get conservation of linked and unlinked peaks from each dataset
p2g_groups <- unique(filt_full_p2gGR$state)

cons_df <- lapply(p2g_groups, function(g){
  sub_p2g_names <- filt_full_p2gGR$peakName[filt_full_p2gGR$state == g] %>% unique()
  lcons <- allPeaksCons$default[allPeaksCons$peakName %in% sub_p2g_names]
  data.frame(
    group=rep(g, times=length(lcons)), 
    conservation=lcons
    )
  }) %>% do.call(rbind,.)

print("success")
# Add conservation of peaks with no link
ulcons <- allPeaksCons$default[allPeaksCons$peakName %ni% filt_full_p2gGR$peakName]
cons_df <- rbind(cons_df, data.frame(
  group=rep("bgd Peaks", times=length(ulcons)),
  conservation=ulcons
  ))

cons_pvals <- lapply(p2g_groups, function(g){
  glcons <- cons_df[cons_df$group == g, "conservation"]
  ulcons <- cons_df[cons_df$group == "bgd Peaks", "conservation"]
  c(linked_mean=mean(glcons, na.rm=TRUE), unlinked_mean=mean(ulcons, na.rm=TRUE), pval=wilcox.test(glcons, ulcons)$p.value)
    # c(linked_mean=mean(glcons, na.rm=TRUE),  pval=wilcox.test(glcons)$p.value)
  }) %>% do.call(rbind,.) %>% as.data.frame()
rownames(cons_pvals) <- p2g_groups

cons_cmap <- c("#EDE3DF","#7FC97F",  "#6A9C89", "#F5E8B7","#FF9B50","#F4AFBD")
names(cons_cmap) <- c("all_cCREs","HSS","HES","MC_level0","MC_level1","MC_level2")
cons_cmap <- c(cons_cmap, unlinked="grey")

# Order by decreasing mean conservation
cons_order <- rownames(cons_pvals[order(cons_pvals$linked_mean, decreasing=TRUE),])
cons_df$group <- factor(cons_df$group, levels=c("bgd Peaks","all_cCREs","HSS","HES","MC_level0","MC_level1","MC_level2"), ordered=TRUE)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 7)
p <- (
  ggplot(cons_df, aes(x=group, y=conservation, fill=group), color="black")
  + geom_boxplot(alpha=1.0)
  + scale_y_continuous(limits=c(0.0,1.05), expand=c(0,0))
  + scale_color_manual(values=cons_cmap)
  + scale_fill_manual(values=cons_cmap)
  + xlab("")
  + ylab("phastCons.100")
  + theme_BOR(border=FALSE)
  + theme(panel.grid.major=element_blank(), 
          panel.grid.minor= element_blank(), 
          plot.margin = unit(c(0.25,1,0.25,1), "cm"), 
          legend.position="none", # Remove legend
          axis.text.x = element_text(angle=45, hjust=1)) 
)
p

pdf(paste0("02.fig2.cCRE_conservation_score.pdf"), width=4, height=5)
p
dev.off()